In [ ]:
!pip install emoji

In [ ]:
import re #For Regular Expression like #,@,$.....
import regex
import numpy as np
import pandas as pd
import emoji #convert Unicode emojis to a string version, and vice versa.
from collections import Counter #counting the occurrences of elements in an iterable
import matplotlib.pyplot as plt
from PIL import Image #Python Imaging Library (PIL) allows you to work with images in various formats
import datetime  #manipulating dates and times
import plotly.express as px #creating complex visualizations easy and intuitive.
from os import path #provides functions for working with file paths
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator # create word clouds, which are visual representations of text data where the size of each word indicates its frequency or importance.stopwords removal (STOPWORDS), color generation based on an image (ImageColorGenerator)

# Preprocess The Data

In [ ]:
# Extract the Date time
def date_time(s):
    pattern='^([0-9]{2})\/([0-9]{2})\/([0-9]{2}), ([0-9]{1,2}):([0-9]{2})\s?(am|pm|AM|PM)? -'
    result=re.match(pattern, s)
    if result:
        return True
    return False

# Extract contacts
def find_contact(s):
    s=s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Extract Message
def getMassage(line):
    splitline=line.split(' - ')
    datetime= splitline[0];
    date, time= datetime.split(', ')
    message=" ".join(splitline[1:])

    if find_contact(message):
        splitmessage=message.split(": ")
        author=splitmessage[0]
        message=splitmessage[1]
    else:
        author=None
    return date, time, author, message

In [68]:
data=[]
conversation='grpchat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer=[]
    date, time, author= None, None, None
    while True:
        line=fp.readline()
        if not line:
            break
        line=line.strip()
        if date_time(line):
            if len(messageBuffer) >0:
                data.append([date, time, author, ''.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message=getMassage(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

# Analyze Sentiment

In [71]:
df=pd.DataFrame(data, columns=["Date", "Time", "contact", "Message"])
df['Date']=pd.to_datetime(df['Date'])

data=df.dropna()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments=SentimentIntensityAnalyzer()
data["positive"]=[sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["negative"]=[sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["neutral"]=[sentiments.polarity_scores(i)["neu"] for i in data["Message"]]

In [ ]:
x=sum(data['positive'])
y=sum(data['negative'])
z=sum(data['neutral'])

def score(a,b,c):
  if(a>b) and (a>c):
    print("Sentiment: Positive")
  elif (b>a) and (b>c):
    print("Sentiment: Negative")
  else:
    print("Sentiment: Neutral")

score(x,y,z)

Sentiment: Negative
